### Davies-Harte Fractional Brownian Motion from Scratch

#### Background Information

In [71]:
import numpy as np

---

**Covariance**

Covariance is a statistical measure of the tendancy of two random variables to move together.

$Cov(X,Y) = \frac{1}{n}\sum_{i=1}^n (X - \mu_X)(Y - \mu_Y) = \mathbb{E}[(X - \mu_X)(Y - \mu_Y)]$

In this context, $X, Y$ are random variables with population means $\mu_X, \mu_Y$ respectively.


---

**Autocovariance Function**

Autocovariance is an implementation of covariance on a time series; specifically, it looks at a random variable's tendancy to move relative to its *lags* (previous points or observations in the time series).

$\gamma_{k} = Cov(X_t, X_{t-k})$ where the covariance can be computed as above.

Here $X_t$ is a time series (ice cream sales, stock price, etc.) where $t = 0, 1, 2, 3, \dots$ it follows naturally, but is needed explicitly in some cases, if the lag exceeds the size of the series (say $N$) then $\gamma_{n} = 0$ $\forall n > N$.

It should be noted that the autocovariance function exhibits symmetry as $\gamma_k = \gamma_{-k}$ which can be easily proven by applying the definition of covariance.


---


**Variance-Covariance & Autocovariance Matrices**

A common representation of data, including time series, is given by a matrix. Features can be expressed in columns where rows represent observations. In a time series or similar panel data, rows typically represent a path (ice cream sales, stock prices, etc.) while columns represent points in time. The Variance-Covariance matrix tells us how features tend together while the Autocovariance matrix is a specific type of Variance-Covariance matrix that computes autocovariances at all relevant lags. 

They are similar structures but are constructed in different ways. There is a simple procedure for generating the autocovariance matrix, a more formal proof and technical discussion can be found: https://medium.com/quant-guild/when-is-the-matrix-product-the-variance-covariance-matrix-ffa047df7d19.


Consider Variance-Covariance matrix $\Sigma$ and data matrix $X$

$\Sigma = \frac{(X - \mu_X)(X - \mu_X)^T}{n-1}$

$\Sigma$ will thus contain a diagonal of variances of each respective feature and covariances of features with each other.

If we apply this methodolgy to a time series we will *not* get the autocovariance matrix as we will be generating point wise covariances - not covariances at general lags. Typically, we do this iteratively for the lags themselves while they are possible.

$\Gamma$ is typically used to denote this autocovariance matrix, consider the following case of lags out to 2...

$$ \Gamma = 
\begin{bmatrix}
\gamma_0 & \gamma_1 & \gamma_2 \\
\gamma_1 & \gamma_0 & \gamma_1 \\
\gamma_2 & \gamma_1 & \gamma_0
\end{bmatrix}
$$

In general,

 $$ \Gamma = 
 \begin{bmatrix}
 \gamma_0 & \gamma_1 & \gamma_2 & \cdots & \gamma_{N-1} \\
 \gamma_1 & \gamma_0 & \gamma_1 & \cdots & \gamma_{N-2} \\
 \gamma_2 & \gamma_1 & \gamma_0 & \cdots & \gamma_{N-3} \\
 \vdots & \vdots & \vdots & \ddots & \vdots \\
 \gamma_{N-1} & \gamma_{N-2} & \gamma_{N-3} & \cdots & \gamma_0
 \end{bmatrix}
 $$


Herein we will consider it the autocovariance matrix as we will be working with time series (Gaussian processes).

---

**Toeplitz Matrices**

A property of *stationary* time series is that their autocovariances are constant over time. In other words, their autocovariance function values will not change throughout the evolution of the process. Unlike the previously defined autocovariance matrix, the Toeplitz matrix *assumes* the top-left to bottom-right diagonals are constant.

 $$ T = 
 \begin{bmatrix}
 \gamma_0 & \gamma_1 & \gamma_2 \\
 \gamma_1 & \gamma_0 & \gamma_1 \\
 \gamma_2 & \gamma_1 & \gamma_0
 \end{bmatrix}
 $$
 
where $\gamma_0, \gamma_1, \gamma_2 \in \mathbb{R}$ are constant real numbers representing the autocovariance at lags 0, 1, and 2 respectively

In terms of referencing the individual elements themselves it isn't uncommon to see notation as $T_{i,j} = \gamma_{i-j}$ where the entry in the ith and jth column produce, in this structure, constant autocovariance $\gamma_{i-j}$.

Grenander, U., & Szegő, G. (1958). *Toeplitz Forms and Their Applications*. University of California Press.

Grenander, U. (1954). *Stochastic Processes and Statistical Inference*. Arkiv för Matematik.


---

**Matrix Diagonalization**

A square matrix (fortunately, all of the matrices that deal with variance-covariance structures are typically square) can be expressed in terms of its eigenvalues and eigenvectors; the so called Eigendecomposition or spectral decomposition. Other requirements are necessary including linearly independent eigenvectors - but we won't worry about these cases for the sake of this article (maybe they will be in the appendix if I can get to it :D).

A square matrix (variance-coveriance matrix, autocovariance matrix) $A \in \mathbb{R}^{n \times n}$ may be represented as $A = P D P^{-1}$ where $D$ is a diagonal matrix containing eigenvalues of $A$, $P$ is a matrix whose columns are eigenvectors of $A$.

Clearly, once a procedure is defined its easy to determine if the solution is correct by taking the product. To diagonalize a matrix we look to solve the *characteristic equation*

$$det(A - \lambda I) = 0$$

where $det(.)$ is the matrix determinate, we first solve for the eigenvalues then for each eigen value $\lambda_i$ we solve 

$$(A - \lambda_i I) = 0$$

where non-zero solutions for $v_i$ yield the corresponding eigenvector to $\lambda_i$

Note: $P$ is a collection of eigenvectors (a matrix) and $D = diag(\lambda)$ where $\lambda = [\lambda_1, \lambda_2, \dots]$ is a vector of eigenvalues placed on the diagonal of a matrix of comparable dimensionality. 


---

**Discrete Fourier Transform**

Quite a fun topic, the Discrete Fourier Transform (DFT) transforms a vector from the time (spatial) domain to the frequency domain.

Suppose $x = [x_0, x_1, \dots, x_{n-1}] \in \mathbb{C}^n$

The DFT is given by

$$\hat{x}_k = \sum_{j=0}^{n-1} x_j e^{\frac{-2 \pi i j k}{n}}$$

which operates fundamentally by changing the basis from standard basis vectors to complex exponentials.

We can invert this transformation from the frequency domain back to the time domain by the inverse DFT (IDFT)

$$x_j = \frac{1}{n}\sum_{k=0}^{n-1} \hat{x}_k e^{\frac{2 \pi i j k}{n}}$$



Cooley, J. W., & Tukey, J. W. (1965). *An Algorithm for the Machine Calculation of Complex Fourier Series*. Mathematics of Computation.

In [72]:
def dft(x : np.ndarray) -> np.ndarray:
    """
    Calculates the Discrete Fourier Transform of a vector x.
    
    The Discrete Fourier Transform (DFT) transforms a vector from the time (spatial) domain to the frequency domain.
    
    Parameters
    ----------
    x : array-like
        Input vector to transform
        
    Returns
    -------
    np.ndarray
        Transformed vector in frequency domain
    """

    xk = np.zeros(len(x), dtype=np.complex128)
    
    for k in range(len(x)):
        for j in range(len(x)):
            xk[k] += x[j] * np.exp(-2j * np.pi * k * j / len(x))
    return xk

def idft(xk : np.ndarray) -> np.ndarray:
    """
    Calculates the Inverse Discrete Fourier Transform of a vector xk.
    
    The Inverse Discrete Fourier Transform (IDFT) transforms a vector from the frequency domain back to the time domain.
    
    Parameters
    ----------
    xk : array-like
        Input vector to transform
        
    Returns
    -------
    np.ndarray
        Transformed vector in time domain
    """

    xj = np.zeros(len(xk), dtype=np.complex128)
    for j in range(len(xk)):
        for k in range(len(xk)):
            xj[j] += xk[k] * np.exp(2j * np.pi * k * j / len(xk))
    return xj / len(xk)

In [ ]:
# time domain to frequency domain and back, confirming the DFT and its inverse IDFT are correct
assert (idft(dft([1, 2, 3, 4, 5])).real.astype(int) == [1, 2, 3, 4, 5]).all()

---

**Discrete Fourier Transform as a Matrix**

The DFT may be viewed as a matrix and the DFT and IDFT as matrix-vector products.

$F \in \mathbb{C}^{n \times n}$ where $F_{j,k} = \omega^{jk}$ and $\omega = e^{\frac{-2 \pi i}{n}}$

Where $F$ is the DFT Matrix, it is common to see many summations represented as matrix-vector products not just avoiding loops but simplifying the overall implementation.

That is $$\hat{x} = Fx$$ and $$x = \frac{1}{n} F^{-1} \hat{x}$$ where $x$ is the vector in the time (spatial) domain and $\hat{x}$ is the vector in the frequency domain.

It is also useful to note $F$ is unitary up to scale with $F^{-1} = \frac{1}{n}F^*$

This means that the DFT matrix $F$ has the special property that its inverse is proportional to its conjugate transpose.

Specifically, $F^{-1} = \frac{1}{n}F^*$ where $F^*$ is the conjugate transpose of $F$ and $n$ is the size of the matrix.

This property makes computing the inverse DFT very efficient since we only need to take the conjugate transpose and scale.

In [ ]:
def dft_matrix(x : np.ndarray) -> np.ndarray:
    """
    Calculates the Discrete Fourier Transform of a vector x.
    
    The Discrete Fourier Transform (DFT) transforms a vector from the time (spatial) domain to the frequency domain.
    
    Parameters
    ----------
    x : array-like
        Input vector to transform
        
    Returns
    -------
    np.ndarray
        Transformed vector in frequency domain
    """

    # the DFT matrix
    F = np.zeros((len(x), len(x)), dtype=np.complex128)

    for k in range(len(x)):
        for j in range(len(x)):
            F[k, j] = np.exp(-2j * np.pi * k * j / len(x))
    
    return F

def idft_matrix(x : np.ndarray) -> np.ndarray:
    """
    Calculates the Inverse Discrete Fourier Transform of a vector x.
    
    The Inverse Discrete Fourier Transform (IDFT) transforms a vector from the frequency domain back to the time domain.
    
    Parameters
    ----------
    x : array-like
        Input vector to transform
        
    Returns
    -------
    np.ndarray
        Transformed vector in time domain
    """

    # DFT matrix
    F = dft_matrix(x)

    # using the property that the inverse matrix is the conjugate of F scaled
    F_star = np.zeros((len(x), len(x)), dtype=np.complex128)
    for k in range(len(x)):
        for j in range(len(x)):
            F_star[k, j] = F[j, k].conjugate()

    return F_star

In [ ]:
# Example implementation of DFT and IDFT as matrix-vector products
x = np.array([1, 2, 3, 4, 5])
F = dft_matrix(x) # DFT matrix
F_star = idft_matrix(x) # IDFT matrix

freq_domain = F @ x

# assess if the matrix-vector product matches the basic DFT implementation
assert np.allclose(freq_domain, dft(x), rtol=1e-10, atol=1e-10)

time_domain = (1 / len(x)) * F_star @ freq_domain

# assess if the matrix-vector product matches the basic IDFT implementation
assert np.allclose(time_domain.real, idft(dft(x)).real, rtol=1e-10, atol=1e-10)

---

**Circulant Matrix**

Quite an appropriate name, the circulant matrix is defined entirely by its first row $C \in \mathbb{R}^{n \times n}$

Each row in the subsequent circulant matrix is a right rotation of the row right above it (shift all the elements to the right by one each step down in the matrix). 

Where $c = [c_0, c_1, \dots, c_{n-1}]$ and fully specifies $C$ it follows

 $$ C = circ(c) =
 \begin{bmatrix}
 c_0 & c_1 & c_2 & \cdots & c_{n-1} \\
 c_{n-1} & c_0 & c_1 & \cdots & c_{n-2} \\
 c_{n-2} & c_{n-1} & c_0 & \cdots & c_{n-3} \\
 \vdots & \vdots & \vdots & \ddots & \vdots \\
 c_1 & c_2 & c_3 & \cdots & c_0
 \end{bmatrix}
 $$

 Now, we can do Eigendecomposition (Cholesky) to acheive the eigenvalues - however this is quite slow in $O(n^3)$.
 
Davis, P. J. (1970). *Circulant Matrices*. Wiley.

---

#### Simulating Random Variables

---

#### Simulating a Gaussian Vector with Covariance Structure

If the goal is to construct a random vector $X$ such that $\mathbb{E} = 0$ with $Cov(X) = \Gamma$

We must find a matrix $A$ such that $\Gamma = AA^T \implies X = AZ$ where $Z \sim N(0, I)$

*Note:* this is quite like the univariate case of $\sigma^2 = a \implies X = \sigma Z$ where $Z \sim N(0, 1)$ and $a \in \mathbb{R}$ is the desired variance of process $X$

There are several ways to accomplish this task:
- Cholesky decomposition
- Square root via eigendecomposition
- Circulant diagonalization (FFT)

*Why eigenvalues?*

The eigenvalues tell us the amount of variance that exists in each principal direction of the Gaussian distribution. More specifically, the Gaussian here ($Z$) is elliptical in high dimensions with shape and spread dictated by eigenvalues (stretch) and eigenvectors (direction).

*Actual Implementation*

1.) Decompose $\Gamma = Q \Lambda Q^T$

2.) Generate $Z \sim N(0, I)$

3.) Set $X = Q \Lambda ^{1/2} Z$ (This is where the eigenvectors $Q$ and scaled eigenvalues $\sqrt{\lambda}$ come in to play)

This should look familiar as if we want the scaling by $\lambda$ in the variance sense we will need to scale by $\sqrt{\lambda}$

*Remark:*

This doesn't work magically...

$$Cov(X) = Cov(Q \Lambda^{1/2} Z) = Q \Lambda^{1/2} \mathbb{E}[ZZ^T] \Lambda^{1/2}Q^T = Q \Lambda^{1/2} I \Lambda^{1/2}Q^T = Q \Lambda Q^T = \Gamma$$

This yields the Gaussian vector $X$ with the desired covariance structure. The "challenge" then becomes determining *what* the covariance structure *should* be and how efficiently can we diagonalize it. The $Cov(Z) = \mathbb{E}[ZZ^T]$ works to compute covariance here as $Z$ is a zero mean process.

---

#### Fractional Gaussian Noise